In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from omnivault.utils.torch_utils.model_utils import total_trainable_parameters, total_parameters
from rich.pretty import pprint

tokenizer = AutoTokenizer.from_pretrained("gpt2")

model = AutoModelForCausalLM.from_pretrained("gpt2")
pprint(total_trainable_parameters(model))



124439808

In [18]:
import torch

torch.sqrt(torch.tensor(175_000_000_000))

tensor(418330.)

In [19]:
175_000_000_000 / (418330 * 8)

52291.253316759496

In [14]:
41800 * 1 * 2

83600

In [4]:
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

# Idea 1. Low intrinsic

-   Low intrinsic: We take inspiration from Li et al. (2018a); Aghajanyan et al.
    (2020) which show that the learned over-parametrized models in fact reside
    on a low intrinsic dimension. We hypothesize that the change in weights
    during model adaptation also has a low “intrinsic rank”, leading to our
    proposed Low-Rank Adaptation (LoRA) approach. LoRA allows us to train some
    dense layers in a neural network indirectly by optimizing rank decomposition
    matrices of the dense layers’ change during adaptation instead, while
    keeping the pre-trained weights frozen, as shown in Figure 1. Using GPT-3
    175B as an example, we show that a very low rank (i.e., r in Figure 1 can be
    one or two) suffices even when the full rank (i.e., d) is as high as 12,288,
    making LoRA both storage- and compute-efficient.

-   Use image from lora paper, it has better latex.

-   LoRA makes training more efficient and lowers the hardware barrier to entry
    by up to 3 times when using adaptive optimizers since we do not need to
    calculate the gradients or maintain the optimizer states for most
    parameters. Instead, we only optimize the injected, much smaller low-rank
    matrices.

    -   This idea is good to know, adam for eg depends on trainable params, and
        now stores less due to lora.

-   Our simple linear design allows us to merge the trainable matrices with the
    frozen weights when deployed, introducing no inference latency compared to a
    fully fine-tuned model, by construction.

    -   Not too sure how it works but I know you can add matrices.

    -   After read below i know exactly how it works. Just distributive law
        because W @ x then if have a gradient update of $\nabla W$ you get
        $(W + \nabla W) @ x$ but note you can do $Wx + \nabla W x$ so $\nabla W$
        is separate. so means you have original weights of pretrained model as
        $W$ and final finetuned weight update to be $\nabla W$ and during
        inference u can add them (merge) but what is attractive is u can store
        nabla W outside of pretrained model and if nabla W is v small then
        hoseh! So nabla W = B @ A here.

    If we paid close attention, the full finetuning and LoRA depictions in the
    figure above look slightly different from the formulas I have shown earlier.
    That's due to the distributive law of matrix multiplication: we don't have
    to add the weights with the updated weights but can keep them separate. For
    instance, if

    x is the input data, then we can write the following for regular finetuning:
    The fact that we can keep the LoRA weight matrices separate makes LoRA
    especially attractive. In practice, this means that we don't have to modify
    the weights of the pretrained model at all, as we can apply the LoRA
    matrices on the fly. This is especially useful if you are considering
    hosting a model for multiple customers. Instead of having to save the large
    updated models for each customer, you only have to save the small set of
    LoRA weights alongside the original pretrained model.

-   Suppose we are given a pre-trained autoregressive language model PΦ(y|x)
    parametrized by Φ. For instance, PΦ(y|x) can be a generic multi-task learner
    such as GPT (Radford et al., b; Brown et al., 2020) based on the Transformer
    architecture (Vaswani et al., 2017). Consider adapting this pre-trained
    model to downstream conditional text generation tasks, such as
    summarization, machine reading comprehension (MRC), and natural language to
    SQL (NL2SQL). Each downstream task is represented by a training dataset of
    context-target pairs: Z = {(xi , yi)}i=1,..,N , where both xi and yi are
    sequences of tokens. For example, in NL2SQL, xi is a natural language query
    and yi its corresponding SQL command; for summarization, xi is the content
    of an article and yi its summary.

    -   here what is the notation phi represent.
    -   also check if classification means y is single target.

-   During full fine-tuning, the model is initialized to pre-trained weights Φ0
    and updated to Φ0 + ∆Φ by repeatedly following the gradient to maximize the
    conditional language modeling objective: max Φ X (x,y)∈Z X |y| t=1 log
    (PΦ(yt|x, y<t)) (1) One of the main drawbacks for full fine-tuning is that
    for each downstream task, we learn a different set of parameters ∆Φ whose
    dimension |∆Φ| equals |Φ0|. Thus, if the pre-trained model is large (such as
    GPT-3 with |Φ0| ≈ 175 Billion), storing and deploying many independent
    instances of fine-tuned models can be challenging, if at all feasible.
    -   here full finetuning drawback is easy, because your weights size and
        grads dont change.
    -   note the dimension |\phi| refers to the total params.
-   Lit review on existing solns, can try skip.
-   section 4.1 is hard.

-   A Generalization of Full Fine-tuning. A more general form of fine-tuning
    allows the training of a subset of the pre-trained parameters. LoRA takes a
    step further and does not require the accumulated gradient update to weight
    matrices to have full-rank during adaptation. This means that when applying
    LoRA to all weight matrices and training all biases2 , we roughly recover
    the expressiveness of full fine-tuning by setting the LoRA rank r to the
    rank of the pre-trained weight matrices. In other words, as we increase the
    number of trainable parameters 3 , training LoRA roughly converges to
    training the original model, while adapter-based methods converges to an MLP
    and prefix-based methods to a model that cannot take long input sequences.

    -   Do not understand.

-   No Additional Inference Latency
-   even though the output dimension is usually sliced into attention heads. We
    limit our study to only adapting the attention weights for downstream tasks
    and freeze the MLP modules (so they are not trained in downstream tasks)
    both for simplicity and parameter-efficiency.W
    -   This just means they just take the concat W matrices and not care about
        each sub-weight matrices.
-   LoRA adds trainable pairs of rank decomposition matrices in parallel to
    existing weight matrices. As mentioned in Section 4.2, we only apply LoRA to
    Wq and Wv in most experiments for simplicity. The number of trainable
    parameters is determined by the rank r and the shape of the original
    weights: |Θ| = 2 × LˆLoRA × dmodel × r, where LˆLoRA is the number of weight
    matrices we apply LoRA to.
-   Low-Rank Structures in Deep Learning. L too deep!
